In [ ]:
#Medium Article:https://medium.com/@abonia/summarization-with-langchain-b3d83c030889
##https://github.com/Abonia1/Langchain-Summarizer/blob/main/Summarization.ipynb

In [1]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pickle
import ollama
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings


import pandas as pd
#from langchain_community.vectorstores import Chroma



import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

#from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
import pickle

In [4]:
pdf_enrichment_output_dir = '../pdf_enrichment/pdf_enriched_output/'  
pdf_enrichment_output_file = 'pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle'

with open(pdf_enrichment_output_dir+pdf_enrichment_output_file, 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [5]:
from langchain.document_loaders import PyPDFLoader
pdf_file="../pdf_parsing/pdf_source/Kubernetes_in_Action.pdf"
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
print(pages[2].page_content)

Kubernetes in Action
 www.allitebooks.com


In [ ]:
full_doc_summary=[]
doc_len=len(document_dict_deserialized)
for i in range(0,doc_len):
    summary=document_dict_deserialized[i]["text"]
    full_doc_summary.append(summary)

In [ ]:
full_doc_summary[11]

In [6]:
#LLM Model for Prompt Tuning
model_name="llama3.1"
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.15)



In [7]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain


question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"])

In [8]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

In [9]:
from time import gmtime, strftime
start_time=strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
start_time=strftime("%Y-%m-%d %H:%M:%S")
print(start_time)
refine_outputs = refine_chain({"input_documents": pages})
end_time=strftime("%Y-%m-%d %H:%M:%S")
print(end_time)

2024-09-13 07:47:51


/home/matrix4284/mambaforge-pypy3/envs/search_agent_poc/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
from pathlib import Path
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [40]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)

In [41]:
pdf_refine_summary

,file_name,file_type,page_number,chunks,concise_summary
0,Kubernetes_in_Action,.pdf,0,MANNINGMarko Lukša \nwww.allitebooks.com,It seems you didn't provide any text for me to...
1,Kubernetes_in_Action,.pdf,1,Kubernetes resources covered in the book\n* Cl...,Here is a concise summary of the text in bulle...
2,Kubernetes_in_Action,.pdf,2,Kubernetes in Action\n www.allitebooks.com,It seems like you forgot to include the actual...
3,Kubernetes_in_Action,.pdf,3,www.allitebooks.com,It seems you forgot to include the actual text...
4,Kubernetes_in_Action,.pdf,4,Kubernetes\nin Action\nMARKO LUKŠA\nMANNING\n...,Here is a concise summary of the text in bulle...
...,...,...,...,...,...
622,Kubernetes_in_Action,.pdf,623,INDEX 592\nstatus\nof nodes 305\npods with unk...,Here is a concise summary of the text in bulle...
623,Kubernetes_in_Action,.pdf,624,INDEX 593\ntopologyKey 471–473\ntriggering\nro...,Here is a concise summary of the text in bulle...
624,Kubernetes_in_Action,.pdf,625,INDEX 594\nvolumes (continued)\nspecification...,Here is a concise summary of the text in bulle...
625,Kubernetes_in_Action,.pdf,626,Kubernetes resources covered in the book (cont...,Here is a concise summary of the text in bulle...


In [42]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_summary = pdf_refine_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the datafram by filename and page_number
pdf_refine_summary.reset_index(inplace=True, drop=True)
pdf_refine_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,Kubernetes_in_Action,.pdf,0,MANNINGMarko Lukša \nwww.allitebooks.com,It seems you didn't provide any text for me to...
1,Kubernetes_in_Action,.pdf,1,Kubernetes resources covered in the book\n* Cl...,Here is a concise summary of the text in bulle...
2,Kubernetes_in_Action,.pdf,2,Kubernetes in Action\n www.allitebooks.com,It seems like you forgot to include the actual...
3,Kubernetes_in_Action,.pdf,3,www.allitebooks.com,It seems you forgot to include the actual text...
4,Kubernetes_in_Action,.pdf,4,Kubernetes\nin Action\nMARKO LUKŠA\nMANNING\n...,Here is a concise summary of the text in bulle...
